<a href="https://colab.research.google.com/github/SomaOkuda/Tweet-Analysis/blob/main/network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install janome
#MeCabインストール
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.996.3
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a
!pip install emoji
!pip install japanize-matplotlib

In [ ]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import community
import copy
import random
from networkx.algorithms.community import greedy_modularity_communities
from networkx.algorithms.community.centrality import girvan_newman
import MeCab
from janome.tokenizer import Tokenizer
tagger=MeCab.Tagger('-Owakati')
t = Tokenizer()
import re
import glob
import collections

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#csv統合
csv_files1 = glob.glob('/content/drive/My Drive/Tweet/tweet/auto_test/*.csv') #jpサンプルツイートフォルダ内結合
data_list1 = []
for file in csv_files1:
  data_list1.append(pd.read_csv(file, lineterminator='\n'))  
df = pd.concat(data_list1, axis=0, sort=False)
df = df.drop_duplicates(keep='last') 
df=df.reset_index(drop=True)

In [ ]:
def Remove_word(list):
  rems = ['@', ',', '，', '。', '', ' ', 'http', 'https','//', '、', '×', '.',
            '+', '＋', 
            '/',', ', '(', ')', '（', '）', '://', 'RT', ':', ';','：', '；',
            '」','「', '＠', '-', '／',  '%', '％', '!', '！', '#', '＃',
            '$', '＄', '=', '＝', 'ー', '￥', '^', '{', '}', '~', '～', 
            '&', '＆', '・', '?', '??', '？', '？？',  '<', '＜', '>', '＞', '_', '＿', 
            'co', 'jp', '(@', '○', '『', '』', '”', '’', '"', '…', 
            ')」', '〇', '【', '】', '[', ']',
            'から', 'より', 'こそ', 'でも', 'しか','さえ', 'けれど', 'たり', 'つつ', 'とも',
            'たら', 'ある', 'なら', 'のに', 'です', 'ます', 'する', 'ほど', 'ない', 'くる',
            'なり', 'そう', 'まし', 'その', 'この', 'あの', 'せる', 'どう', 'ため', 'どこ',
            'いる', 'これ', 'それ', 'あれ', 'いい', 'など', 'あっ', 'もう', 'さん', 'じゃ',
            'から', 'あり', 'ので', 'とも', 'ませ', 'でし', 'とき', 'こと', 'なる', 'って',
            'ただ', 'まで', 'もの', 'つか', 'なっ', 'でき', 'もっ', 'けど', 'ほぼ', 'なー',
            'そこ', 'ここ', 'だろ', 'なん', 'だっ', 'なあ', 'っけ', 'せる', 'やっ', 'また',
            'どれ', 'なれ', 'かも', 'いく', 'いけ', 'いう', 'たい', 'あと', 'かも', 'しれ',  
            'こう', 'なく', 'よく', 'だけ', 'れる', 'よう', 'かけ', 'どの', 'てる', 'とか',
            'という', '思う', '思っ', 'として', 'ところ', 'しまう', 'なんか', 'そんな', 
            'でしょ', 'しまい', 'わかり', 'まま'
            ]
  for number in range(10000):
    rems.append(str(number))

# 一文字のひらがなとカタカナ，全角数字，アルファベット大文字・小文字
  hirakana    = [chr(i) for i in range(12353, 12436)]
  katakana    = [chr(i) for i in range(12449, 12533)]
  zenkaku_num = [chr(i) for i in range(65296, 65296+10)]
  alph_L      = [chr(i) for i in range(97, 97+26)]
  alph_S      = [chr(i) for i in range(65, 65+26)]

  for a in range(len(hirakana)):
    rems.append(hirakana[a])
  for b in range(len(katakana)):
    rems.append(katakana[b])
  for c in range(len(zenkaku_num)):
    rems.append(zenkaku_num[c])
  for d in range(len(alph_L)):
    rems.append(alph_L[d])
  for e in range(len(alph_S)):
    rems.append(alph_S[e])

  words = [x for x in list if (x not in rems)]

  return words

In [ ]:
mention_from=[]
mention_message=[]
mention_to=[]
for i in range(len(df)):
  if str(df.iloc[i]['text']).startswith('RT'):
    try:
      text=df.iloc[i]['text']
      result = re.findall("(?<![@\w])@(\w{1,25})", text)
      if result[0]:
        try:
          mention_from.append(df.iloc[i]['user_name'])#user_name=screen_name
          mention_message.append(text)
          mention_to.append(result[0]) #RTの場合、本質的にRTされたツイートを保持するユーザーとのリンクを考える
          #mention_to.append(''.join(result)) #メンションの場合はこれだけでOK ⇨ 文中のメンション及び＠から始まる反応を取れていない（が普通に文中に＠を使うような絵文字等が出てくるとできない）
        except:
          pass
    except:
      pass

In [ ]:
df_rt=pd.DataFrame({'from': mention_from, 'to': mention_to, 'message': mention_message})

In [ ]:
#コロナ関連のツイートに関するRTネットワーク
covid_num=[]
for i in range(len(df_rt)):
  text=df_rt.iloc[i]['message']
  word_list=list(tagger.parse(text).split())
  if len([word for word in word_list if word in ['ワクチン']]) !=0:
    covid_num.append(i)

df_covid=df_rt.iloc[covid_num]
df_covid=df_covid.reset_index(drop=True)
len(df_covid)

In [ ]:
#networkx: https://networkx.org/documentation/stable//reference/drawing.html
import time
import networkx as nx
G=nx.Graph()
for i in range(len(df_covid)):
    from_user=df_covid.iloc[i]['from']
    to_user=df_covid.iloc[i]['to']
    G.add_edge(from_user, to_user, weight = 1)
#d=1のものを除去
#https://networkx.org/documentation/stable/reference/classes/generated/networkx.Graph.remove_node.html
#2以上を取る手法はこれ：http://faculty.washington.edu/kstarbi/examining-trolls-polarization.pdf
for n, d in list(G.degree()):
  if d<=2:
    G.remove_node(n)

In [ ]:
nx.write_gml(G,'/content/drive/My Drive/Tweet/gml/220110_vaccine_RT_dgt2.gml')

In [ ]:
import matplotlib.pyplot as plt
G = nx.read_gml('/content/drive/My Drive/Tweet/gml/220110_vaccine_RT_dgt2.gml')

In [ ]:
#RTネットワークの大きいものを分析
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G0 = G.subgraph(Gcc[0])

In [ ]:
#two=community.kernighan_lin_bisection(G0)
#参考: https://networkx.org/documentation/stable/reference/algorithms/community.html
from networkx.algorithms import community
comp=community.centrality.girvan_newman(G0)
#two=community.kernighan_lin_bisection(G0)
two=list(sorted(c) for c in next(comp))
cluster_number=[]
for node in G0.nodes():
  for i in range(len(two)):
    if node in two[i]:
      cluster_number.append(i)

#d = dict(zip(G0.nodes(), cluster_number))
#nx.set_node_attributes(G0, name='cluster', values=d)
#nx.write_gml(G0,'/content/drive/MyDrive/Income_prediction/0420_network/modularity_over2degree_corona_negpos_rt_network.gml')

In [ ]:
d = dict(zip(G0.nodes(), cluster_number))
nx.set_node_attributes(G0, name='cluster', values=d)
nx.write_gml(G0,'/content/drive/My Drive/Tweet/gml/211227_vaccine_clustered_dgt3.gml')

In [ ]:
c = collections.Counter(cluster_number)
print(c)

#propagationによるcommunityの特定
#参考：https://www.programmersought.com/article/19431905738/

In [ ]:
from networkx.algorithms import community
communities_generator = list(community.label_propagation_communities(G))

In [ ]:
from networkx.algorithms import community
def label_propagation_community(G):
    communities_generator = list(community.label_propagation_communities(G))
    m = []
    for i in communities_generator:
        m.append(list(i))
    return m

In [ ]:
g=label_propagation_community(G)

cluster_number=[]
for node in G.nodes():
  for i in range(len(g)):
    if node in g[i]:
      cluster_number.append(i)

In [ ]:
max=0
max_number=0
for i in range(len(g)):
  if len(g[i])>max:
    max=len(g[i])
    max_number=i

In [ ]:
d = dict(zip(G.nodes(), cluster_number))
nx.set_node_attributes(G, name='cluster', values=d)
nx.write_gml(G,'/content/drive/My Drive/Tweet/gml/220110_vactine_propagation_RT_dgt2.gml')